## Using SOMA to label unlabeled mocaps

We roughly base this off of the two SOMA tutorials: [Tutorial 1](https://github.com/nghorbani/soma/blob/main/src/tutorials/run_soma_on_soma_dataset.ipynb) and [Tutorial 2](https://github.com/nghorbani/soma/blob/main/src/tutorials/label_priming.ipynb).

Reading the [definitions](https://github.com/nghorbani/soma/tree/main/src/tutorials#definitions) of terminology is recommended.

We will be using a pre-trained SOMA model called the "SuperSet" to label our unlabeled motioin capture data. The SuperSet trains a body model with 89 markers placed around the body, and thus will label our unlabeled mocap data by matching our unlabeled markers to a subset of the 89 markers. If we have an unlabeled marker that does not correspond to a marker on the SuperSet, the behavior is undefined -- either that marker will be unlabeled (marked as ghost points/discarded), or it will be labelled with some nearby marker label. This imperfect labelling can lead to downstream errors when trying to fit the SMPL model to the labelled mocap data.

It is possible to train a new SOMA model, for a different marker layout (specifically a marker layout that does not fit a subset of the SuperSet), please check this [README](https://github.com/nghorbani/soma/tree/main/src/tutorials/README.md) and Tutorial 1 for information on this. As per the note in Tutorial 1, the AMASS marker noise model and the CAESAR parameters both help the model training, but neither are publicly available due to licensing. Thus it is more optimal to use the pretrained models. As such, we will not be covering model training in this tutorial.

Conceptually, this is how the SOMA training works (see [this paper](https://arxiv.org/pdf/2110.04431.pdf) for more details):
- Goal: to label unlabeled mocap data for a specific *markerlayout*
- First, we take a single frame from a single trial of our unlabeled mocap data
  - one frame needed per subject, since each subject has a unique body -- and thus unique markerlayouts/point clouds -- thus a unique body model
- This frame is labelled
- This labelled frame is MoSHed to get a SMPL-X body model for that specific subject
- virtual markers corresponding to our markerlayout are automotically placed on this SMPL-X body
- This SMPL body model is animated by AMASS motions
  - AMASS dataset has many actions saved in SMPL format, which allows us to have our subject-specific body-model 'perform' many actions
- Thus a large repository of labeled mocap data is synthesized, data which is specific to our subject and our markerlayout
- Real noise data is added to this synthetic data
- occlusions and ghost points also added
- SOMA model is trained on this synthetic data, where the model is given the input of a synthetic unlabeled point cloud and learns to output the corresponding labels
- Transformers multiple self-attention units are used to learn the spatial structure of the 3d body, as well as an optimal transport layer to encode the natural constraints of the human body.